## json parsing

In [189]:
df.to_excel('/workspace/task6/data_all.xlsx',index=False)

In [2]:
from glob import glob
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import os
import csv
import json
from collections import OrderedDict
base_path = '/workspace/task6/04_최종_xml'
base_save_path = '/workspace/task6/04_최종_라벨링데이터'
clinical_info_path = '/workspace/task6/1205_clinical_info.csv'
df = pd.read_csv(clinical_info_path)
# 결측값 처리1
df = df.fillna(-9999999)
# assert len(glob(base_path+'/*.xml')) == len(df)
df

,Task_Name,Center,Image_no,Patient_no,Case_no,Slide_no,Age,Sex,Procedure,Location1,Location2,Invasiveness,LN meta,Distant meta,Pathologist
0,6,1,1,1,1,1,60,F,Resection,HEAD_NECK,Face,Invasive,Yes,No,11
1,6,1,2,1,1,2,60,F,Resection,HEAD_NECK,Face,Invasive,Yes,No,11
2,6,1,3,1,1,3,60,F,Resection,HEAD_NECK,Face,Invasive,Yes,No,11
3,6,1,4,1,1,4,60,F,Resection,HEAD_NECK,Face,Invasive,Yes,No,11
4,6,1,5,1,1,5,60,F,Resection,HEAD_NECK,Face,Invasive,Yes,No,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,6,4,1167,536,1,1,62,M,Biopsy,EXTREMITY,Sole,In situ,NS,No,11
3138,6,4,1168,537,1,1,69,F,Biopsy,EXTREMITY,Sole,In situ,NS,No,11
3139,6,4,1169,538,1,1,65,M,Biopsy,EXTREMITY,Sole,In situ,NS,No,11
3140,6,4,1170,539,1,1,44,M,Biopsy,EXTREMITY,Palm,In situ,NS,No,11


In [ ]:
#########final 파싱 #########

except_list=[]
execpt_list_1=[]
for row_ in df.iterrows():
    case_dict = dict(row_[1])
    case_dict = {k : None if case_dict[k] == -9999999 else v for k, v in case_dict.items()}
    task_ = '{:02d}'.format(case_dict['Task_Name'])
    center_ = '{:02d}'.format(case_dict['Center'])
    Image_no_ ='{:05d}'.format(case_dict['Image_no'])
    patient_no_ = '{:04d}'.format(case_dict['Patient_no'])
    case_no_ ='{:05d}'.format(case_dict['Case_no'])
    fname_ = f'{task_}_{center_}_{Image_no_}'
    fname_1= f'{task_}_{center_}'
    PROCEDURE_ = case_dict['Procedure']
    SEX_ = case_dict['Sex']
    AGE_ = int(case_dict['Age'])
    LOCATION1_ = case_dict['Location1']
    LOCATION2_ = case_dict['Location2']
    INVASIVENESS_= case_dict['Invasiveness']
    LN_meta_= case_dict['LN meta']
    Distant_meta_ = case_dict['Distant meta']  
    PATHOLOGIST_ = '{:02d}'.format(case_dict['Pathologist'])
    Clinical_information = OrderedDict(HOSP=center_, PATIENT_NO=patient_no_,CASE_NO=case_no_, AGE=AGE_,SEX=SEX_,
                                       PROCEDURE=PROCEDURE_,LOCATION1=LOCATION1_,LOCATION2=LOCATION2_,INVASIVENESS=INVASIVENESS_,
                                       LN_meta=LN_meta_,Distant_meta=Distant_meta_,PATHOLOGIST=PATHOLOGIST_)

    xml_path_ = os.path.join(base_path,'06',fname_1,fname_,fname_+'.xml')
    json_path_ = os.path.join(base_path,'06',fname_1,fname_,fname_+'.json')
    if json_path in 
    print(xml_path_)
    
    try:
        if os.path.isfile(xml_path_):
            tree = ET.parse(xml_path_)
            root = tree.getroot()
            Layer1 = []
            Layer2 = []
            # aperio image scope
            if len(root.attrib):
                # layer 1 : 대표 검체, layer2 : 병변 
                assert len(list(root.iter('Annotation'))) == 2,f"{fname_}.xml does not match the ID range, len {len(list(root.iter('Annotation')))}"
                for root_ in (root.iter('Annotation')):
                    for anno_ in (root_.iter('Region')):
                        rect_ = []
                        for child in anno_.iter("Vertex"):
                            coodinates = child.attrib
                            x_ = int(round(float(coodinates.get('X'))))
                            y_ = int(round(float(coodinates.get('Y'))))
                            rect_.append({"X": x_, "Y":y_})
                        if root_.attrib['Id'] == '1':
                            Layer1.append({'Region' : rect_,'NegativeROA' : int(anno_.attrib['NegativeROA'])})
                        elif root_.attrib['Id'] == '2':
                            Layer2.append({'Region' : rect_,'NegativeROA' : int(anno_.attrib['NegativeROA'])})
                        else:
                            raise Exception(f'{fname_} is out of ID format')
            # ASAP
            else:
                for root_ in root.iter('Annotation'):
                    g_n = root_.attrib['PartOfGroup']
                    if g_n[-1] == 'P':
                        nega_pen = 0
                    elif g_n[-1] == 'N':
                        nega_pen = 1
                    else:
                        raise Exception

                    rect_ = []
                    for anno in root_.iter('Coordinate'):
                        coodinates = anno.attrib
                        x_ = int(round(float(coodinates.get('X'))))
                        y_ = int(round(float(coodinates.get('Y'))))
                        rect_.append({"X": x_, "Y":y_})
                    if g_n[:2] =='L1':
                        Layer1.append({'Region' : rect_,'NegativeROA' : nega_pen})
                    elif g_n[:2] =='L2':
                        Layer2.append({'Region' : rect_,'NegativeROA' : nega_pen})
                    else:
                        raise Exception(f'{fname_} is out of ID format')

            result_ = OrderedDict(Clinical_information=Clinical_information,Layer1=Layer1,Layer2=Layer2)
    #     print(result_)
    #     else:
    #         raise Exception(f'No such {fname_}.xml in {base_path}')

            if os.path.isdir(base_save_path):
                save_json_path_ = f'{base_save_path}/{task_}/{task_}_{center_}/{task_}_{center_}_{Image_no_}'
                print(save_json_path_)
                os.makedirs(save_json_path_,exist_ok=True)

                save_json_path_ = os.path.join(save_json_path_,fname_+'.json')
                with open(save_json_path_, "w", encoding='UTF-8-sig') as json_:
                    json_.write(json.dumps(result_, indent=4,sort_keys=False))  
            else :
                execpt_list_1.append(xml_path_)
    except:
        except_list.append(xml_path_)
        continue

In [ ]:
except_list=[]
execpt_list_1=[]
for row_ in df.iterrows():
    case_dict = dict(row_[1])
    case_dict = {k : None if case_dict[k] == -9999999 else v for k, v in case_dict.items()}
    task_ = '{:02d}'.format(case_dict['Task_Name'])
    center_ = '{:02d}'.format(case_dict['Center'])
    Image_no_ ='{:05d}'.format(case_dict['Image_no'])
    patient_no_ = '{:04d}'.format(case_dict['Patient_no'])
    case_no_ ='{:05d}'.format(case_dict['Case_no'])
    fname_ = f'{task_}_{center_}_{Image_no_}'
    fname_1= f'{task_}_{center_}'
    PROCEDURE_ = case_dict['Procedure']
    SEX_ = case_dict['Sex']
    AGE_ = int(case_dict['Age'])
    LOCATION1_ = case_dict['Location1']
    LOCATION2_ = case_dict['Location2']
    INVASIVENESS_= case_dict['Invasiveness']
    LN_meta_= case_dict['LN meta']
    Distant_meta_ = case_dict['Distant meta']  
    PATHOLOGIST_ = '{:02d}'.format(case_dict['Pathologist'])
    Clinical_information = OrderedDict(HOSP=center_, PATIENT_NO=patient_no_,CASE_NO=case_no_, AGE=AGE_,SEX=SEX_,
                                       PROCEDURE=PROCEDURE_,LOCATION1=LOCATION1_,LOCATION2=LOCATION2_,INVASIVENESS=INVASIVENESS_,
                                       LN_meta=LN_meta_,Distant_meta=Distant_meta_,PATHOLOGIST=PATHOLOGIST_)

    xml_path_ = os.path.join(base_path,'06',fname_1,fname_,fname_+'.xml')
    print(xml_path_)
    try:
        if os.path.isfile(xml_path_):
            tree = ET.parse(xml_path_)
            root = tree.getroot()
            Layer1 = []
            Layer2 = []
            # aperio image scope
            if len(root.attrib):
                # layer 1 : 대표 검체, layer2 : 병변 
                assert len(list(root.iter('Annotation'))) == 2,f"{fname_}.xml does not match the ID range, len {len(list(root.iter('Annotation')))}"
                for root_ in (root.iter('Annotation')):
                    for anno_ in (root_.iter('Region')):
                        rect_ = []
                        for child in anno_.iter("Vertex"):
                            coodinates = child.attrib
                            x_ = int(round(float(coodinates.get('X'))))
                            y_ = int(round(float(coodinates.get('Y'))))
                            rect_.append({"X": x_, "Y":y_})
                        if root_.attrib['Id'] == '1':
                            Layer1.append({'Region' : rect_,'NegativeROA' : int(anno_.attrib['NegativeROA'])})
                        elif root_.attrib['Id'] == '2':
                            Layer2.append({'Region' : rect_,'NegativeROA' : int(anno_.attrib['NegativeROA'])})
                        else:
                            raise Exception(f'{fname_} is out of ID format')
            # ASAP
            else:
                for root_ in root.iter('Annotation'):
                    g_n = root_.attrib['PartOfGroup']
                    if g_n[-1] == 'P':
                        nega_pen = 0
                    elif g_n[-1] == 'N':
                        nega_pen = 1
                    else:
                        raise Exception

                    rect_ = []
                    for anno in root_.iter('Coordinate'):
                        coodinates = anno.attrib
                        x_ = int(round(float(coodinates.get('X'))))
                        y_ = int(round(float(coodinates.get('Y'))))
                        rect_.append({"X": x_, "Y":y_})
                    if g_n[:2] =='L1':
                        Layer1.append({'Region' : rect_,'NegativeROA' : nega_pen})
                    elif g_n[:2] =='L2':
                        Layer2.append({'Region' : rect_,'NegativeROA' : nega_pen})
                    else:
                        raise Exception(f'{fname_} is out of ID format')

            result_ = OrderedDict(Clinical_information=Clinical_information,Layer1=Layer1,Layer2=Layer2)
    #     print(result_)
    #     else:
    #         raise Exception(f'No such {fname_}.xml in {base_path}')

            if os.path.isdir(base_save_path):
                save_json_path_ = f'{base_save_path}/{task_}/{task_}_{center_}/{task_}_{center_}_{Image_no_}'
                print(save_json_path_)
                os.makedirs(save_json_path_,exist_ok=True)

                save_json_path_ = os.path.join(save_json_path_,fname_+'.json')
                with open(save_json_path_, "w", encoding='UTF-8-sig') as json_:
                    json_.write(json.dumps(result_, indent=4,sort_keys=False))  
            else :
                execpt_list_1.append(xml_path_)
    except:
        except_list.append(xml_path_)
        continue

## json parsing except_list

In [13]:
except_list

['/workspace/task6/04_최종_xml/06/06_01/06_01_00341/06_01_00341.xml',
 '/workspace/task6/04_최종_xml/06/06_01/06_01_00734/06_01_00734.xml',
 '/workspace/task6/04_최종_xml/06/06_02/06_02_00168/06_02_00168.xml',
 '/workspace/task6/04_최종_xml/06/06_02/06_02_00233/06_02_00233.xml',
 '/workspace/task6/04_최종_xml/06/06_02/06_02_00235/06_02_00235.xml',
 '/workspace/task6/04_최종_xml/06/06_03/06_03_00073/06_03_00073.xml',
 '/workspace/task6/04_최종_xml/06/06_03/06_03_00076/06_03_00076.xml',
 '/workspace/task6/04_최종_xml/06/06_03/06_03_00079/06_03_00079.xml',
 '/workspace/task6/04_최종_xml/06/06_03/06_03_00126/06_03_00126.xml',
 '/workspace/task6/04_최종_xml/06/06_03/06_03_00209/06_03_00209.xml',
 '/workspace/task6/04_최종_xml/06/06_04/06_04_00045/06_04_00045.xml',
 '/workspace/task6/04_최종_xml/06/06_04/06_04_00572/06_04_00572.xml',
 '/workspace/task6/04_최종_xml/06/06_04/06_04_00867/06_04_00867.xml',
 '/workspace/task6/04_최종_xml/06/06_04/06_04_00868/06_04_00868.xml',
 '/workspace/task6/04_최종_xml/06/06_04/06_04_0107

In [10]:
result_

OrderedDict([('Clinical_information',
              OrderedDict([('HOSP', '04'),
                           ('PATIENT_NO', '0540'),
                           ('CASE_NO', '00001'),
                           ('AGE', 48),
                           ('SEX', 'F'),
                           ('PROCEDURE', 'Biopsy'),
                           ('LOCATION1', 'EXTREMITY'),
                           ('LOCATION2', 'Sole'),
                           ('INVASIVENESS', 'In situ'),
                           ('LN_meta', 'NS'),
                           ('Distant_meta', 'No'),
                           ('PATHOLOGIST', '11')])),
             ('Layer1',
              [{'Region': [{'X': 34887, 'Y': 96082},
                 {'X': 27261, 'Y': 96463},
                 {'X': 22114, 'Y': 102754},
                 {'X': 20970, 'Y': 110951},
                 {'X': 20970, 'Y': 118768},
                 {'X': 24783, 'Y': 125440},
                 {'X': 32409, 'Y': 126965},
                 {'X': 39271, 'Y': 

## schema validation

In [ ]:
from jsonschema import validate
from pathlib import Path
import json
import tqdm
from glob import glob
insitu_except_list=[]
in_path_list=[]
TASK_NUM = "06"
BASE_PATH = "/workspace/task6/04_최종_라벨링데이터"
jsonfile_path_list = glob(f"{BASE_PATH}/{TASK_NUM}/*/*/*.json")
jsonfile_path_list
schema_path = f"/workspace/task6/task{TASK_NUM}_schema.json"
with open (schema_path, "r", encoding="utf-8-sig") as schemafile:
    schema_data = json.load(schemafile)
    
for path in tqdm.tqdm(jsonfile_path_list):
    with open (path, "r", encoding="utf-8-sig") as jsonfile:
        json_data = json.load(jsonfile)
    try :
        validate(instance=json_data, schema=schema_data)
        in_path_list.append(path)
    except Exception as e :
        insitu_except_list.append(path)
#         damn.append(e.message)
#         assholes.append(jsonfile)
        name_ = path.split('/')[-1].replace('.json','')
        print(f"{name_} : {e.message}")

In [4]:
len(insitu_except_list)

951

In [35]:
df3=df.loc[df['Invasiveness'] == 'Invasive']

In [182]:
tmp_1209

,Task_Name,Center,Image_no,Patient_no,Case_no,Slide_no,Age,Sex,Procedure,Location1,Location2,Invasiveness,LN meta,Distant meta,Pathologist
0,6,1,1,1,1,1,60,F,Resection,HEAD_NECK,Face,Invasive,Yes,No,11
1,6,1,2,1,1,2,60,F,Resection,HEAD_NECK,Face,Invasive,Yes,No,11
2,6,1,3,1,1,3,60,F,Resection,HEAD_NECK,Face,Invasive,Yes,No,11
3,6,1,4,1,1,4,60,F,Resection,HEAD_NECK,Face,Invasive,Yes,No,11
4,6,1,5,1,1,5,60,F,Resection,HEAD_NECK,Face,Invasive,Yes,No,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3059,6,4,1080,466,1,1,48,F,Resection,EXTREMITY,NS,Invasive,NS,No,11
3060,6,4,1081,467,1,1,34,M,Biopsy,EXTREMITY,Lower leg,Invasive,NS,No,11
3061,6,4,1082,468,1,1,37,F,Biopsy,EXTREMITY,Foot dorsum,Invasive,NS,No,11
3062,6,4,1083,469,1,1,66,M,Biopsy,EXTREMITY,Sole,Invasive,NS,No,11


In [ ]:
base_path = '/workspace/task6/04_최종_라벨링데이터'
invasive_test_path=[]
for row_ in tmp_1209.iterrows():
    case_dict = dict(row_[1])
    case_dict = {k : None if case_dict[k] == -9999999 else v for k, v in case_dict.items()}
    task_ = '{:02d}'.format(case_dict['Task_Name'])
    center_ = '{:02d}'.format(case_dict['Center'])
    Image_no_ ='{:05d}'.format(case_dict['Image_no'])
    patient_no_ = '{:04d}'.format(case_dict['Patient_no'])
    case_no_ ='{:05d}'.format(case_dict['Case_no'])
    fname_ = f'{task_}_{center_}_{Image_no_}'
    fname_1= f'{task_}_{center_}'
    INVASIVENESS_= case_dict['Invasiveness']
    json_path_ = os.path.join(base_path,'06',fname_1,fname_,fname_+'.json')
    invasive_test_path.append(json_path_)
invasive_test_path

In [186]:
pd.DataFrame(invasive_test_path).to_csv('test_final_1209')

In [ ]:
base_path = '/workspace/task6/04_최종_라벨링데이터'
invasive_train_path=[]
for row_ in df4.iterrows():
    case_dict = dict(row_[1])
    case_dict = {k : None if case_dict[k] == -9999999 else v for k, v in case_dict.items()}
    task_ = '{:02d}'.format(case_dict['Task_Name'])
    center_ = '{:02d}'.format(case_dict['Center'])
    Image_no_ ='{:05d}'.format(case_dict['Image_no'])
    patient_no_ = '{:04d}'.format(case_dict['Patient_no'])
    case_no_ ='{:05d}'.format(case_dict['Case_no'])
    fname_ = f'{task_}_{center_}_{Image_no_}'
    fname_1= f'{task_}_{center_}'
    INVASIVENESS_= case_dict['Invasiveness']
    json_path_ = os.path.join(base_path,'06',fname_1,fname_,fname_+'.json')
    invasive_train_path.append(json_path_)
invasive_train_path

## patch extraction

In [ ]:
import json
import os
from glob import glob
from tqdm import tqdm
import openslide
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
LEVEL_IDX ={
    'level0' : 0,
    'level2' : 1,
    'level4' : 2,
}
LEVEL_RESOLUTION = {
    'level0' : 1.0,
    'level1' : 2.0, # 1/2 resoltion
    'level2' : 4.0, # 1/4 resolution
    'level3' : 8.0, # 1/4 resolution
    'level4' : 16.0, # 1/16 resolution
}
PATCH_SIZE=  512
EXTRACTION_LEVEL = 'level2'
svs_base_path = '/workspace/task6/'
save_base_path = '/workspace/task6/04_final_patch'
# ''/workspace/task6/03_중간단계/06/06_02/06_02_00363/06_02_00363'

json_path_list=invasive_train_path
# json_path_list = in_path_list  ##schema validation 완료된 파일만 불러오기
# print(len(json_path_list))
name_list = []
except_list = []
except_list_json= []
df_ex=pd.DataFrame(columns=['slide name','N patches'])
for path_ in tqdm(sorted(json_path_list[13:])):
    name_ = path_.split('/')[-1].replace('.json','')
    name_list.append(name_)
    #slide setting
    # 이부분 수정해야됨
    base_slide_path = path_.replace('04_최종_라벨링데이터','04_원천데이터').replace('.json','')
    
#     is_svs = False
    try:
        if os.path.isfile(base_slide_path+'.svs') == True:           
            wsi_slide = openslide.OpenSlide(base_slide_path+'.svs')           
            mpp = round(float(wsi_slide.properties['aperio.MPP']),2)
        elif os.path.isfile(base_slide_path+'.mrxs') == True:
            wsi_slide = openslide.OpenSlide(base_slide_path+'.mrxs')
            mpp = round(float(wsi_slide.properties['openslide.mpp-x']),2)
        elif os.path.isfile(base_slide_path+'.tif') == True:
            wsi_slide = openslide.OpenSlide(base_slide_path+'.tif')
            mpp = round(float(wsi_slide.properties['openslide.comment'].split('MPP=')[-1]),2)        
        else:
            raise Exception('{} 해당 경로의 파일 없음.'.format(base_slide_path))
        print(base_slide_path)
    except:        
        except_list.append(base_slide_path)
        print('ex',base_slide_path)
        continue
    level_list = [round(i) for i in wsi_slide.level_downsamples]
    level_idx = level_list.index(LEVEL_RESOLUTION[EXTRACTION_LEVEL])
    
#     if os.path.isfile(svs_path) == False:
#         raise Exception('{} 해당 경로의 파일 없음.'.format(svs_path))
#     wsi_slide = openslide.OpenSlide(svs_path)
    #check mpp
    try:
        if mpp > 0.40:
            wsi_w,wsi_h = wsi_slide.level_dimensions[level_idx]
            wsi_array = np.array(wsi_slide.read_region([0,0], level_idx, [wsi_w, wsi_h]))[:,:,:3]
            MPP_W = 1
            print(wsi_array.shape)
        else:
            wsi_w,wsi_h = wsi_slide.level_dimensions[level_idx]
            wsi_array = np.array(wsi_slide.read_region([0,0], level_idx, [wsi_w, wsi_h]))[:,:,:3]
            wsi_array = cv2.resize(wsi_array, dsize=(0,0),fx =0.5,fy = 0.5, interpolation=cv2.INTER_AREA)
            wsi_h,wsi_w = wsi_array.shape[:-1]
            print(wsi_h,wsi_w)
            MPP_W = 2
    except:
        except_list.append(base_slide_path)
        print('ex',base_slide_path)
        continue  
        
        #json setting
    try:
        with open (path_, "r", encoding="utf-8-sig") as jsonfile:
            dict_ = json.load(jsonfile)
            layer_1= dict_['Layer1']
            layer_2= dict_['Layer2']
    except:
        except_list_json.append(base_slide_path)
        print('ex',base_slide_path)
        continue         
    #layer1
    layer1_mat_posi = np.zeros((wsi_h,wsi_w))
    layer1_mat_nega = np.zeros((wsi_h,wsi_w))
    
    #foreground만 추출
    wsi_array_gray = cv2.cvtColor(wsi_array,cv2.COLOR_RGB2GRAY)
    th_, otsu_result =  cv2.threshold(wsi_array_gray, 240, 255, cv2.THRESH_BINARY_INV) 
    kernel = np.ones((25,25),np.uint8)
    foreground = cv2.morphologyEx(otsu_result, cv2.MORPH_CLOSE, kernel)
    for region_ in layer_1:
        bound_ = []
        for cord_ in region_['Region']:
            x_,y_ = cord_['X']//int(LEVEL_RESOLUTION[EXTRACTION_LEVEL]*MPP_W), cord_['Y']//int(LEVEL_RESOLUTION[EXTRACTION_LEVEL]*MPP_W)
            bound_.append([x_,y_])
        cnt_ = np.array(bound_).reshape((-1, 1, 2)).astype(np.int32)
        if region_['NegativeROA']== 0:
                cv2.fillPoly(layer1_mat_posi, [cnt_], 1)
        elif region_['NegativeROA']== 1:
            cv2.fillPoly(layer1_mat_nega, [cnt_], 1)
    layer1_mat_posi[foreground==0] = 0
    layer1_mat_posi[layer1_mat_nega==1] = 0
    #layer2
    layer2_mat_posi = np.zeros((wsi_h,wsi_w))
    layer2_mat_nega = np.zeros((wsi_h,wsi_w))
    for region_ in layer_2:
        bound_ = []
        for cord_ in region_['Region']:
            x_,y_ = cord_['X']//int(LEVEL_RESOLUTION[EXTRACTION_LEVEL]*MPP_W), cord_['Y']//int(LEVEL_RESOLUTION[EXTRACTION_LEVEL]*MPP_W)
            bound_.append([x_,y_])
        cnt_ = np.array(bound_).reshape((-1, 1, 2)).astype(np.int32)
        if region_['NegativeROA']== 0:
                cv2.fillPoly(layer2_mat_posi, [cnt_], 1)
        elif region_['NegativeROA']== 1:
            cv2.fillPoly(layer2_mat_nega, [cnt_], 1)
    layer2_mat_posi[layer2_mat_nega==1] = 0
    if np.sum(layer1_mat_posi)== 0 :
        continue
                
#102911:23             
#     #json setting -> 좌표 추출
#     with open (path_, "r", encoding="utf-8-sig") as jsonfile:
#         dict_ = json.load(jsonfile)
#         layer_1= dict_['Layer1']
#         layer_2= dict_['Layer2']
#     #layer1
#     layer1_mat_posi = np.zeros((wsi_h,wsi_w))
#     layer1_mat_nega = np.zeros((wsi_h,wsi_w))
#     for region_ in layer_1:
#         bound_ = []
#         for cord_ in region_['Region']:
#             x_,y_ = cord_['X']//int(4.0*MPP_W), cord_['Y']//int(4.0*MPP_W)
#             bound_.append([x_,y_])
#         cnt_ = np.array(bound_).reshape((-1, 1, 2)).astype(np.int32)
#         if region_['NegativeROA']== 0:
#                 cv2.fillPoly(layer1_mat_posi, [cnt_], 1)
#         elif region_['NegativeROA']== 1:
#             cv2.fillPoly(layer1_mat_nega, [cnt_], 1)
#     layer1_mat_posi[layer1_mat_nega==1] = 0
#     #layer2
#     layer2_mat_posi = np.zeros((wsi_h,wsi_w))
#     layer2_mat_nega = np.zeros((wsi_h,wsi_w))
#     for region_ in layer_2:
#         bound_ = []
#         for cord_ in region_['Region']:
#          #x,y는 level0에서 그려지므로, 위에서 우리는 img를 resize했으므로, 좌표도 똑같이 나눠준다.
#             x_,y_ = cord_['X']//int(4.0*MPP_W), cord_['Y']//int(4.0*MPP_W)
#             bound_.append([x_,y_])
#         cnt_ = np.array(bound_).reshape((-1, 1, 2)).astype(np.int32)
#         #positive.영역 안 그리기
#         if region_['NegativeROA']== 0:
#                 cv2.fillPoly(layer2_mat_posi, [cnt_], 1)
#         #negagive 영역 안 그리기
#         elif region_['NegativeROA']== 1:
#             cv2.fillPoly(layer2_mat_nega, [cnt_], 1)
# # #     if is_svs ==True:
# #     plt.imshow(layer1_mat_posi)
# #     plt.show()
# #     plt.imshow(layer1_mat_nega)
# #     plt.show()
# #     plt.imshow(layer2_mat_posi)
# #     plt.show()
# #     plt.imshow(layer2_mat_nega)
# #     plt.show()
# #     else:
# #         continue
# #     print(np.unique(layer1_mat_posi))
    count=0
    print(wsi_h)
    for y0 in range(0,wsi_h,PATCH_SIZE*3):
        for x0 in range(0,wsi_w,PATCH_SIZE*3):         
            if not ((y0+PATCH_SIZE) > wsi_h) or ((x0+PATCH_SIZE) > wsi_w):
                core_ = np.uint8(layer1_mat_posi[y0:y0+PATCH_SIZE,x0:x0+PATCH_SIZE])
                mask_ = np.uint8(layer2_mat_posi[y0:y0+PATCH_SIZE,x0:x0+PATCH_SIZE])
                patch_ = np.uint8(wsi_array[y0:y0+PATCH_SIZE,x0:x0+PATCH_SIZE])

                if np.sum(core_)/(PATCH_SIZE**2) > 0.7:
                    cv2.imwrite(os.path.join(save_base_path,'level2_patch_1205',f'{name_}_X0_{int(x0*4.0*MPP_W)}_Y0_{int(y0*4.0*MPP_W)}.png'),cv2.cvtColor(patch_,cv2.COLOR_RGB2BGR))
                    cv2.imwrite(os.path.join(save_base_path,'level2_mask_1205',f'{name_}_X0_{int(x0*4.0*MPP_W)}_Y0_{int(y0*4.0*MPP_W)}.png'),mask_)
                    count+=1
            else:
                continue
    df_ex=df_ex.append(pd.DataFrame([[path_.split('/')[-2],count]],columns=['slide name','N patches']))
    print(count)
name_df = pd.DataFrame(name_list,columns= ['name'])
name_df.to_csv('tvt_df_1206_invasive_1.csv',index=False)

#     plt.imshow(layer1_mat_posi)
#     plt.show()
#     plt.imshow(layer2_mat_posi)
#     plt.show()

In [14]:
df_ex.to_csv('invasive_Npatches.csv') # slide별 patch수 저장
tmp_1=glob('/workspace/task6/04_final_patch/level2_patch_1205/*')
print('total_patches:',len(tmp_1))

## train

In [154]:
slide_tvt=pd.concat([e['name'],e['tvt'].sample(frac=1).reset_index(drop=True)], axis=1)

In [159]:
# tvt_df = pd.read_csv('/workspace/task6/tvt_df.csv')
tvt_df=df
base_path = '/workspace/task6/04_final_patch/level2_patch_1205'
DEVICE = 'cuda'
BATCH_SIZE = [16,1]
best_loss = 1000
ls_cnt = 0
epochs = 1
LR = 0.00001
def get_path_list(tvt_df,key,base_path):
    name_list = tvt_df.loc[tvt_df['tvt']==key,'name'].tolist()
    path_list = []
    for name_ in tqdm(name_list):
        path_list += glob(os.path.join(base_path,name_+'*.png'))
    return path_list

train_path_list = get_path_list(slide_tvt,'train',base_path)
valid_path_list = get_path_list(slide_tvt,'valid',base_path)
# test_path_list = get_path_list(slide_tvt,'test',base_path)
print(len(train_path_list))
print(len(valid_path_list))
# print(len(test_path_list))

100% 2479/2479 [01:33<00:00, 26.50it/s]
100% 301/301 [00:11<00:00, 27.21it/s]

34696
3890


In [131]:
base_patch_list='/workspace/task6/04_final_patch/level2_patch_1205/*'

In [133]:
train_=glob(base_patch_list)

38107

In [163]:
import pandas as pd
import os
from glob import glob
import torch
from torch import nn
import torchvision
import os
from torch.nn import functional as F
import albumentations as A
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import parser
from torchvision import transforms, utils
import cv2
from torch.nn import Sequential
from albumentations.pytorch.transforms import ToTensorV2
import pandas as pd
from  tqdm import tqdm
import matplotlib.pyplot as plt
import random
import copy
from datetime import datetime

import segmentation_models_pytorch as smp

import numpy as np
import torchvision.models as models

In [161]:
def get_augmentation():
    _transform = [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
    ]
    return A.Compose(_transform)

def get_preprocessing():
    _transform = [
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]
    return A.Compose(_transform)

In [164]:
class dataset(nn.Module):
    
    def __init__(self, d, augmentation=None, preprocessing=None):

        
        self.image_path_li = d
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, idx):
        img = cv2.imread(self.image_path_li[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  
        
        mask = cv2.imread(self.image_path_li[idx].replace('level2_patch_1205','level2_mask_1205'),0)
        
        if self.augmentation:
                sample = self.augmentation(image=img, mask=mask)
                img, mask = sample['image'], sample['mask']

        if self.preprocessing:
            sample = self.preprocessing(image=img, mask=mask)
            img, mask = sample['image'], sample['mask']
              
        return img, mask.unsqueeze(0)
    
    def __len__(self):
        return len(self.image_path_li) 

In [165]:
train_dataset = dataset(train_path_list,augmentation = get_augmentation(),preprocessing = get_preprocessing())
valid_dataset = dataset(valid_path_list,preprocessing = get_preprocessing())
# test_dataset = dataset(test_,preprocessing = get_preprocessing())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE[0], shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE[1], shuffle=False, num_workers=0)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

In [166]:
seg_model = smp.Unet(
    encoder_name = 'efficientnet-b7',        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet", # use `imagenet` pretrained weights for encoder initialization
    activation = 'sigmoid',
    in_channels=3,                  # model input channels (1 for grayscale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
    )

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /home/tester/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


  0%|          | 0.00/254M [00:00<?, ?B/s]

In [167]:
loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
    smp.utils.metrics.Fscore(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=seg_model.parameters(), lr=LR),
])


# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    seg_model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)
valid_epoch = smp.utils.train.ValidEpoch(
    seg_model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [173]:
!nvidia-smi

Wed Dec  7 07:27:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.04    Driver Version: 455.23.04    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    On   | 00000000:01:00.0 Off |                  N/A |
| 30%   37C    P8    20W / 350W |  24196MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    On   | 00000000:25:00.0 Off |                  N/A |
| 55%   

In [174]:
os.environ["CUDA_VISIBLE_DEVICES"]= '1'

In [ ]:
history_train_loss = []
history_valid_loss = []
history_train_iou = []
history_valid_iou = []
for i in range(epochs):
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    history_train_loss.append(train_logs['dice_loss'])
    history_valid_loss.append(valid_logs['dice_loss'])
    history_train_iou.append(train_logs['iou_score'])
    history_valid_iou.append(valid_logs['iou_score'])
    if best_loss > valid_logs['dice_loss']:
        model_state = copy.deepcopy(seg_model.state_dict())
        best_loss = valid_logs['dice_loss']
        ls_cnt=0
    else:
        ls_cnt+=1
        if ls_cnt>10:
            torch.save({'model_state' : model_state,
                    'history_dict': {'train_loss' : history_train_loss,'train_iou' : history_train_iou,
                                    'valid_loss' : history_valid_loss,'valid_iou' : history_valid_iou}},
                    f'/workspace/task6/model_final_1207/model{ls_cnt}.pth')
        if ls_cnt ==10:
            print('early stop')
            break 
torch.save({'model_state' : model_state,
            'history_dict': {'train_loss' : history_train_loss,'train_iou' : history_train_iou,
                            'valid_loss' : history_valid_loss,'valid_iou' : history_valid_iou}},
            'model_final_1207.pth')

In [169]:
!cd model_final_1207